# Import

This notebook was created with reference to https://www.kaggle.com/code/jaejohn/eedi-bge-starter 

I trained the retriever on train set.

Please let me know if there are any mistakes.

I think more inprovements should be made. I apologize for the unorganized and messy code.

In [ ]:
#!pip install -q sentence_transformers

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity

import re
from scipy.spatial.distance import cdist

import gc, torch
def flush():
    gc.collect()
    torch.cuda.empty_cache()
    
device = torch.device("cuda")

train = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv")
test = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv")
misconception_mapping = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")


In [ ]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

def make_all_question_text(df: pd.DataFrame) -> pd.DataFrame:
    df["all_question_text"] = df["ConstructName"] + " " + df["QuestionText"]
    df["all_question_text"] = df["all_question_text"].apply(preprocess_text)
    return df

def wide_to_long_train_ans(df: pd.DataFrame) -> pd.DataFrame:
    df = pd.melt(
        df[
            [
                "QuestionId",
                "all_question_text",
                "CorrectAnswer",
                "AnswerAText",
                "AnswerBText",
                "AnswerCText",
                "AnswerDText",
                #"MisconceptionAId",
                #"MisconceptionBId",
                #"MisconceptionCId",
                #"MisconceptionDId"
            ]
        ],
        id_vars    = ["QuestionId", "all_question_text", "CorrectAnswer"], 
        var_name   = 'Answer',
        value_name = 'ans_value'
    )
    return df

def wide_to_long_train_mis(df: pd.DataFrame) -> pd.DataFrame:
    df = pd.melt(
        df[
            [
                "QuestionId",
                "all_question_text",
                "CorrectAnswer",
                #"AnswerAText",
                #"AnswerBText",
                #"AnswerCText",
                #"AnswerDText",
                "MisconceptionAId",
                "MisconceptionBId",
                "MisconceptionCId",
                "MisconceptionDId"
            ]
        ],
        id_vars    = ["QuestionId", "all_question_text", "CorrectAnswer"],      
        var_name   = 'Misconception',
        value_name = 'mis_value'
    )
    return df

def make_all_text_train(df: pd.DataFrame) -> pd.DataFrame:
    text_components = []
    if "all_question_text" in df.columns:
        text_components.append(df["all_question_text"])
    if "ans_value" in df.columns:
        text_components.append(df["ans_value"].apply(preprocess_text))
    
    df["all_text"] = pd.concat(text_components, axis=1).apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
    return df

In [ ]:
train = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv")
train = make_all_question_text(train)
train_long_ans = make_all_text_train(wide_to_long_train_ans(train))
train_long_mis = make_all_text_train(wide_to_long_train_mis(train))
train_long_ans["Id"] = list(range(len(train_long_ans)))
train_long_mis["Id"] = list(range(len(train_long_mis)))
train_long = pd.merge(train_long_ans, train_long_mis, on=["QuestionId", "Id", "all_question_text", "CorrectAnswer"], how="left")
train_long = train_long.sort_values(["QuestionId", "Misconception"]).reset_index(drop=True)
train_long.loc[3, "all_text_x"]

In [ ]:
train_long_drop = train_long.dropna(subset=["mis_value"])
train_long_drop["mis_value"] = train_long_drop["mis_value"].astype(int)
train_long_drop

In [ ]:
#from sentence_transformers import SentenceTransformer, InputExample
#from sentence_transformers import models, losses
#from torch.utils.data import DataLoader

In [ ]:
#model_retriever = AutoModelForSequenceClassification.from_pretrained(
#    '/kaggle/input/bge-small-en-v1.5/transformers/bge/2', 
#    num_labels=len(misconception_mapping['MisconceptionName']))
#tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/bge-small-en-v1.5/transformers/bge/2')
#model_name = '/kaggle/input/bge-small-en-v1.5/transformers/bge/2'
#word_embedding_model = models.Transformer(model_name)
#pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
#model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# Define a list with sentences (1k - 100k sentences)
#train_sentences = list(train_long_drop["all_text_x"])

# Convert train sentences to sentence pairs
#train_data = [InputExample(texts=[s, s]) for s in train_sentences]

# DataLoader to batch your data
#train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)

# Use the denoising auto-encoder loss
#train_loss = losses.MultipleNegativesRankingLoss(model)

# Call the fit method
#model.fit(
#    train_objectives=[(train_dataloader, train_loss)], epochs=1, show_progress_bar=True, use_amp=True
#)

#model.save("output/simcse-model")
#model_retriever.to(device)

In [ ]:
#from datasets import Dataset

#train_long_drop = train_long.dropna(subset=["mis_value"])
#train_long_drop["mis_value"] = train_long_drop["mis_value"].astype(int)
#train_long_drop

# train_model
#train_data = Dataset.from_pandas(train_long_drop)
# train_data["mis_value"][10]

#def preproc(example):
#    encoded_example = tokenizer(example["all_text_x"], max_length=1024)
#    encoded_example["labels"] = example["mis_value"]
#    return encoded_example

#encoded_train_data = train_data.map(
#    preproc,
#    remove_columns = train_data.column_names
#)

#encoded_train_data

In [ ]:
#from transformers import DataCollatorWithPadding

#data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
#from transformers import Trainer, TrainingArguments

#training_args = TrainingArguments(
#    output_dir="/kaggle/working/logs",
#    per_device_train_batch_size=32,
#    fp16=True,learning_rate=1e-4,
#    lr_scheduler_type="cosine",
#    report_to="none",
#    logging_strategy="steps",
#    logging_steps=50,
#)

#trainer = Trainer(
#    model=model_retriever,
#    tokenizer=tokenizer,
#    train_dataset = encoded_train_data,
#    data_collator=data_collator,
#    args=training_args,
#)

#trainer.train()

In [ ]:
#trainer.save_model()

In [ ]:
#text = ["this is a text"]
#inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", max_length=1024).to(device)
#with torch.no_grad():
#    outputs = model_retriever(**inputs, output_hidden_states=True)
#outputs.hidden_states[-1].shape

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity

import re
from scipy.spatial.distance import cdist

# Data Loading
train = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv")
test = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv")
misconception_mapping = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")

# Load the model and tokenizer
device = "cuda:0" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/eedi-simcse-simple-baseline-train/output/simcse-model')
model = AutoModel.from_pretrained('/kaggle/input/eedi-simcse-simple-baseline-train/output/simcse-model')
model.to(device)

# Function to generate embeddings
def generate_embeddings(texts, model, tokenizer, device, batch_size=8):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt", max_length=1024).to(device)
        with torch.no_grad():
            outputs = model(**inputs, output_hidden_states=True)
        embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token
        embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
        all_embeddings.append(embeddings.cpu().numpy())
    return np.concatenate(all_embeddings, axis=0)

# Generate embeddings for misconceptions
MisconceptionName = list(misconception_mapping['MisconceptionName'].values)
all_ctx_vector = generate_embeddings(MisconceptionName, model, tokenizer, device)

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

# Prepare test data
def make_all_question_text(df: pd.DataFrame) -> pd.DataFrame:
    df["all_question_text"] = df["ConstructName"] + " " + df["QuestionText"]
    df["all_question_text"] = df["all_question_text"].apply(preprocess_text)
    return df

test = make_all_question_text(test)

def wide_to_long(df: pd.DataFrame) -> pd.DataFrame:
    df = pd.melt(
        df[
            [
                "QuestionId",
                "all_question_text",
                "CorrectAnswer",
                "AnswerAText",
                "AnswerBText",
                "AnswerCText",
                "AnswerDText"
            ]
        ],
        id_vars    = ["QuestionId", "all_question_text", "CorrectAnswer"],
        var_name   = 'Answer',
        value_name = 'value'
    )
    return df

test_long = wide_to_long(test)

def make_all_text(df: pd.DataFrame) -> pd.DataFrame:
    text_components = []
    if "all_question_text" in df.columns:
        text_components.append(df["all_question_text"])
    if "value" in df.columns:
        text_components.append(df["value"].apply(preprocess_text))
    
    df["all_text"] = pd.concat(text_components, axis=1).apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
    return df

test_long = make_all_text(test_long)
test_long = test_long.sort_values(["QuestionId", "Answer"]).reset_index(drop=True)

# Generate embeddings for test data
test_texts = list(test_long['all_text'].values)
all_text_vector = generate_embeddings(test_texts, model, tokenizer, device)

# Compute similarities
cosine_similarities = cosine_similarity(all_text_vector, all_ctx_vector)

# Euclidean distance
euclidean_distances = cdist(all_text_vector, all_ctx_vector, metric='euclidean')
euclidean_similarities = 1 / (1 + euclidean_distances)  # Convert distance to similarity

# Combination of cosine and euclidean
combined_similarities = (cosine_similarities + euclidean_similarities) / 2

# Use the combined_similarities for sorting
test_sorted_indices = np.argsort(-combined_similarities, axis=1)

# Prepare submission
test_long["Answer_alphabet"] = test_long["Answer"].str.extract(r'Answer([A-Z])Text$')
test_long["QuestionId_Answer"] = test_long["QuestionId"].astype("str") + "_" + test_long["Answer_alphabet"]
test_long["MisconceptionId"] = test_sorted_indices[:, :25].tolist()
test_long["MisconceptionId"] = test_long["MisconceptionId"].apply(lambda x: ' '.join(map(str, x)))
# filter correct row
test_long = test_long[test_long["CorrectAnswer"] != test_long["Answer_alphabet"]]
submission = test_long[["QuestionId_Answer", "MisconceptionId"]].reset_index(drop=True)

In [ ]:
submission.head(10)

In [ ]:
submission.to_csv("submission.csv", index=False)
print("Submission file created successfully!")

In [ ]:
!cat submission.csv